# Tidal Analysis

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import folium
from pathlib import Path
from matplotlib.cm import ScalarMappable
import matplotlib as mpl
from branca.colormap import linear

In [ ]:
BASE = Path("/Users/kyledorman/data/planet_coverage/points_30km/")
FIG_DIR = BASE.parent / "figs" / "simulated_tidal"
FIG_DIR.mkdir(exist_ok=True, parents=True)

ca_ocean = gpd.read_file(BASE / "ca_ocean.geojson")
all_grids_df = gpd.read_file(BASE / "ocean_grids.gpkg")
tide_df = pd.read_csv(BASE / "simulated_tidal_coverage.csv")
heuristics_df = pd.read_csv(BASE / "simulated_tidal_coverage_heuristics.csv")

for col in tide_df.columns:
    tide_df.loc[tide_df[col].isna(), col] = 365.0
    assert not tide_df[col].isna().any()

tide_heuristics_grid_df = tide_df.merge(heuristics_df, on='cell_id').merge(all_grids_df, on='cell_id', how='left')
tide_heuristics_grid_df = gpd.GeoDataFrame(tide_heuristics_grid_df, geometry="geometry")
tide_heuristics_pts_df = tide_heuristics_grid_df.copy()
tide_heuristics_pts_df.geometry = tide_heuristics_grid_df.centroid
tide_heuristics_plot_df = tide_heuristics_grid_df.to_crs(ca_ocean.crs)

tide_heuristics_grid_df.head(5)

In [ ]:
# Create the base map centered on the calculated location
centroid = tide_heuristics_plot_df.geometry.iloc[0].centroid
base_map = folium.Map(location=[centroid.y, centroid.x], zoom_start=2, width=1000, height=800)

for geo in tide_heuristics_plot_df.geometry:
    folium.GeoJson(
        geo,
    ).add_to(base_map)
base_map

In [ ]:
def plot_column(df, column_name, title):
    # --- Folium map for % ---
    if df[column_name].max() == df[column_name].min():
        scale_min = 0
    else:
        scale_min = df[column_name].min()
    color_scale = linear.viridis.scale(scale_min, df[column_name].max())

    centroid = df.geometry.iloc[3000].centroid
    m = folium.Map(
        location=[centroid.y, centroid.x], 
        zoom_start=2, 
        tiles="CartoDB positron",
        width=1000,
        height=800
    )
    
    for _, row in df.iterrows():
        value = row[column_name]
        geom = row.geometry
        folium.GeoJson(
            data=geom,
            style_function=lambda f, col=color_scale(value): {
                "fillColor": col,
                "color":     col,      # outline same as fill
                "weight":    1,
                "fillOpacity": 0.7,
            },
            tooltip=f"{row.cell_id}: {value:0.1f}",
        ).add_to(m)
    
    color_scale.caption = title
    color_scale.add_to(m)

    return m

In [ ]:
plot_column(tide_heuristics_plot_df, 'tide_range', "Tidal Range")

In [ ]:
# ───────────────────────────────────────────────────────────────
# 2. axes layout: rows = sensors, cols = metrics
# ───────────────────────────────────────────────────────────────
sensors       = ["planet", "sentinel", "landsat"]
base_metrics  = ["count"]
metrics       = [f"{lvl}_{m}" for m in base_metrics for lvl in ("low", "high")]

nrows, ncols  = len(sensors), len(metrics)
fig, axes = plt.subplots(
    nrows=nrows,
    ncols=ncols,
    figsize=(ncols * 4, nrows * 2),
    constrained_layout=True,
)

# ───────────────────────────────────────────────────────────────
# 3.  loop over columns (metrics) to set a shared scale per column
# ───────────────────────────────────────────────────────────────
cmap = "viridis"

for c, metric in enumerate(metrics):
    # shared vmin/vmax across sensors for this metric
    col_values = [f"{sat}_{metric}" for sat in sensors]
    vmin = tide_heuristics_pts_df[col_values].min().min()
    vmax = tide_heuristics_pts_df[col_values].max().max()
    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

    for r, sensor in enumerate(sensors):
        ax      = axes[r, c]
        colname = f"{sensor}_{metric}"

        tide_heuristics_pts_df.plot(
            column     = colname,
            ax         = ax,
            cmap       = cmap,
            norm       = norm,
            marker     = "o",
            markersize = 1,
            linewidth  = 0,
            legend     = False,
        )

        # titles: top row gets metric title; first column gets sensor label
        ax.set_title(f"{sensor.title()} {metric.replace('_', ' ').title()}")
        ax.axis("off")

# add ONE colour‑bar for the whole column
sm = ScalarMappable(norm=norm, cmap=cmap);  sm.set_array([])
cax = fig.colorbar(sm, ax=axes[:, -1], shrink=0.6, pad=0.02, location="right")
cax.ax.set_ylabel(base_metrics[0].replace('_', ' ').title())

plt.savefig(FIG_DIR / "tide_count.png", dpi=300)
plt.show()

In [ ]:
# ───────────────────────────────────────────────────────────────
# 2. axes layout: rows = sensors, cols = metrics
# ───────────────────────────────────────────────────────────────
sensors       = ["planet", "sentinel", "landsat"]
base_metrics  = ["count"]
metrics       = [f"{lvl}_{m}" for m in base_metrics for lvl in ("low", "high")]

nrows, ncols  = len(sensors), len(metrics)
fig, axes = plt.subplots(
    nrows=nrows,
    ncols=ncols,
    figsize=(ncols * 4, nrows * 2),
    constrained_layout=True,
)

# ───────────────────────────────────────────────────────────────
# 3.  loop over columns (metrics) to set a shared scale per column
# ───────────────────────────────────────────────────────────────
cmap = "viridis"

for c, metric in enumerate(metrics):
    # shared vmin/vmax across sensors for this metric
    col_values = [f"{sat}_{metric}" for sat in sensors]
    vmin = tide_heuristics_pts_df[col_values].min().min()
    vmax = 1
    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

    for r, sensor in enumerate(sensors):
        ax      = axes[r, c]
        colname = f"{sensor}_{metric}"
        assert not tide_heuristics_pts_df[colname].isna().any()

        tide_heuristics_pts_df.plot(
            column     = colname,
            ax         = ax,
            cmap       = cmap,
            norm       = norm,
            marker     = "o",
            markersize = 1,
            linewidth  = 0,
            legend     = False,
        )

        # titles: top row gets metric title; first column gets sensor label
        ax.set_title(f"{sensor.title()} {metric.replace('_', ' ').title()}")
        ax.axis("off")

# add ONE colour‑bar for the whole column
sm = ScalarMappable(norm=norm, cmap=cmap);  sm.set_array([])
cax = fig.colorbar(sm, ax=axes[:, -1], shrink=0.6, pad=0.02, location="right")
cax.ax.set_ylabel(base_metrics[0].replace('_', ' ').title())

plt.savefig(FIG_DIR / "tide_count_binary.png", dpi=300)
plt.show()

In [ ]:
df = tide_heuristics_pts_df[tide_heuristics_pts_df.tide_range > 2.0]

# ───────────────────────────────────────────────────────────────
# 2. axes layout: rows = sensors, cols = metrics
# ───────────────────────────────────────────────────────────────
sensors       = ["planet", "sentinel", "landsat"]
base_metrics  = ["count"]
metrics       = [f"{lvl}_{m}" for m in base_metrics for lvl in ("low", "high")]

nrows, ncols  = len(sensors), len(metrics)
fig, axes = plt.subplots(
    nrows=nrows,
    ncols=ncols,
    figsize=(ncols * 4, nrows * 2),
    constrained_layout=True,
)

# ───────────────────────────────────────────────────────────────
# 3.  loop over columns (metrics) to set a shared scale per column
# ───────────────────────────────────────────────────────────────
cmap = "viridis"

for c, metric in enumerate(metrics):
    # shared vmin/vmax across sensors for this metric
    col_values = [f"{sat}_{metric}" for sat in sensors]
    vmin = df[col_values].min().min()
    vmax = 1
    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

    for r, sensor in enumerate(sensors):
        ax      = axes[r, c]
        colname = f"{sensor}_{metric}"
        assert not df[colname].isna().any()

        df.plot(
            column     = colname,
            ax         = ax,
            cmap       = cmap,
            norm       = norm,
            marker     = "o",
            markersize = 1,
            linewidth  = 0,
            legend     = False,
        )

        # titles: top row gets metric title; first column gets sensor label
        low_high = metric.split("_")[0]
        ax.set_title(f"{sensor.title()} {low_high.title()} Binary - Large Tidal Range")
        ax.axis("off")

# add ONE colour‑bar for the whole column
sm = ScalarMappable(norm=norm, cmap=cmap);  sm.set_array([])
cax = fig.colorbar(sm, ax=axes[:, -1], shrink=0.6, pad=0.05, location="right")
cax.ax.set_ylabel(base_metrics[0].replace('_', ' ').title())

plt.savefig(FIG_DIR / "tide_count_binary_large_range.png", dpi=300)
plt.show()

In [ ]:
# ───────────────────────────────────────────────────────────────
# 2. axes layout: rows = sensors, cols = metrics
# ───────────────────────────────────────────────────────────────
sensors       = ["planet", "sentinel", "landsat"]
base_metrics  = ["days_between_p95"]
metrics       = [f"{lvl}_{m}" for m in base_metrics for lvl in ("low", "high", "mid")]

nrows, ncols  = len(sensors), len(metrics)
fig, axes = plt.subplots(
    nrows=nrows,
    ncols=ncols,
    figsize=(ncols * 4, nrows * 2),
    constrained_layout=True,
)

# ───────────────────────────────────────────────────────────────
# 3.  loop over columns (metrics) to set a shared scale per column
# ───────────────────────────────────────────────────────────────
cmap = "viridis"

for c, metric in enumerate(metrics):
    # shared vmin/vmax across sensors for this metric
    col_values = [f"{sat}_{metric}" for sat in sensors]
    vmin = 0 # tide_heuristics_pts_df[col_values].min().min()
    vmax = tide_heuristics_pts_df[col_values].max().max()
    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

    for r, sensor in enumerate(sensors):
        ax      = axes[r, c]
        colname = f"{sensor}_{metric}"
        assert not tide_heuristics_pts_df[colname].isna().any(), colname

        tide_heuristics_pts_df.plot(
            column     = colname,
            ax         = ax,
            cmap       = cmap,
            norm       = norm,
            marker     = "o",
            markersize = 1,
            linewidth  = 0,
            legend     = False,
        )

        # titles: top row gets metric title; first column gets sensor label
        low_high = metric.split("_")[0]
        ax.set_title(f"{sensor.title()} {low_high.title()} Days Between p95 - Large Tidal Range")
        ax.axis("off")

# add ONE colour‑bar for the whole column
sm = ScalarMappable(norm=norm, cmap=cmap);  sm.set_array([])
cax = fig.colorbar(sm, ax=axes[:, -1], shrink=0.6, pad=0.02, location="right")
cax.ax.set_ylabel(base_metrics[0].replace('_', ' ').title())

plt.savefig(FIG_DIR / "tide_days_between_p95.png", dpi=300)
plt.show()

In [ ]:
df = tide_heuristics_pts_df[tide_heuristics_pts_df.tide_range > 2.0]

# ───────────────────────────────────────────────────────────────
# 2. axes layout: rows = sensors, cols = metrics
# ───────────────────────────────────────────────────────────────
sensors       = ["planet", "sentinel", "landsat"]
base_metrics  = ["days_between_p95"]
metrics       = [f"{lvl}_{m}" for m in base_metrics for lvl in ("low", "high", "mid")]

nrows, ncols  = len(sensors), len(metrics)
fig, axes = plt.subplots(
    nrows=nrows,
    ncols=ncols,
    figsize=(ncols * 4, nrows * 2),
    constrained_layout=True,
)

# ───────────────────────────────────────────────────────────────
# 3.  loop over columns (metrics) to set a shared scale per column
# ───────────────────────────────────────────────────────────────
cmap = "viridis"

for c, metric in enumerate(metrics):
    # shared vmin/vmax across sensors for this metric
    col_values = [f"{sat}_{metric}" for sat in sensors]
    vmin = 0 # df[col_values].min().min()
    vmax = df[col_values].max().max()
    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

    for r, sensor in enumerate(sensors):
        ax      = axes[r, c]
        colname = f"{sensor}_{metric}"
        assert not df[colname].isna().any(), colname

        df.plot(
            column     = colname,
            ax         = ax,
            cmap       = cmap,
            norm       = norm,
            marker     = "o",
            markersize = 1,
            linewidth  = 0,
            legend     = False,
        )

        # titles: top row gets metric title; first column gets sensor label
        ax.set_title(f"{sensor.title()} {metric.replace('_', ' ').title()}")
        ax.axis("off")

# add ONE colour‑bar for the whole column
sm = ScalarMappable(norm=norm, cmap=cmap);  sm.set_array([])
cax = fig.colorbar(sm, ax=axes[:, -1], shrink=0.6, pad=0.02, location="right")
cax.ax.set_ylabel(base_metrics[0].replace('_', ' ').title())

plt.savefig(FIG_DIR / "tide_days_between_p95_large_range.png", dpi=300)
plt.show()

In [ ]:
sensors       = ["planet", "sentinel", "landsat"]
base_metrics  = ["count"]
metrics       = [f"{lvl}_{m}" for m in base_metrics for lvl in ("low", "high", "mid")]

for metric in metrics:
    print(metric, "% No observations")
    for sensor in sensors:
        print(sensor, round(100 * (tide_df[f'{sensor}_{metric}'] == 0).sum() / len(tide_df), 1))

In [ ]:
df = tide_heuristics_grid_df[tide_heuristics_grid_df.tide_range > 2.0]
sensors       = ["planet", "sentinel", "landsat"]
base_metrics  = ["count"]
metrics       = [f"{lvl}_{m}" for m in base_metrics for lvl in ("low", "high", "mid")]

for metric in metrics:
    print(metric, "% No observations - Large Range")
    for sensor in sensors:
        print(sensor, round(100 * (df[f'{sensor}_{metric}'] == 0).sum() / len(tide_df), 1))

In [ ]:
sensors       = ["planet", "sentinel", "landsat"]
base_metrics  = ["days_between_p95"]
metrics       = [f"{lvl}_{m}" for m in base_metrics for lvl in ("low", "high", "mid")]

for metric in metrics:
    print(metric, "% > 100 days")
    for sensor in sensors:
        print(sensor, round(100 * (tide_df[f'{sensor}_{metric}'] > 100).sum() / len(tide_df), 1))

In [ ]:
sensors       = ["planet", "sentinel", "landsat"]
base_metrics  = ["days_between_p95"]
metrics       = [f"{lvl}_{m}" for m in base_metrics for lvl in ("low", "high", "mid")]

for metric in metrics:
    print(metric, "median")
    for sensor in sensors:
        print(sensor, round(tide_df[f'{sensor}_{metric}'].median(), 1))

In [ ]:
df = tide_heuristics_grid_df[tide_heuristics_grid_df.tide_range > 2.0]
sensors       = ["planet", "sentinel", "landsat"]
base_metrics  = ["days_between_p95"]
metrics       = [f"{lvl}_{m}" for m in base_metrics for lvl in ("low", "high", "mid")]

for metric in metrics:
    print(metric, "median - Large Range")
    for sensor in sensors:
        print(sensor, round(df[f'{sensor}_{metric}'].median(), 1))

In [ ]:
tide_heuristics_grid_df.tide_range.hist()